In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
from datetime import datetime, timedelta
import pymysql
from selenium.webdriver.support.ui import WebDriverWait

In [5]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
chrome = webdriver.Chrome('./chromedriver',chrome_options=options)
data = {}
tStart = time.time()
for i in range(1,2):
    chrome.get(url.format(i,i))
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card = soup.find_all(class_="a-size-medium a-color-base a-text-normal")  
    data.setdefault('card',[]).append([ele.text for ele in amazon_card])
chrome.close() 
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

C:\Users\a4619\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


總共耗時:2.277080 秒


In [40]:
# 資料庫設定
db_settings = {
    "host": "127.0.0.2",
    "port": 3306,
    "user": "root",
    "password": "0531",
    "db": "python",
    "charset": "utf8"
}

In [44]:
search_date = datetime.strftime(datetime.now() , '%Y-%m-%d')
try:
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
        for adx,datas in enumerate(data['card']):
            for every_data in datas:                
                matchObj = re.split(r'\s', every_data, re.M|re.I)
                insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s,%s,%s);"    
                try:
                    cursor.execute(insert,(matchObj[0],every_data,adx+1,search_date))
                except Exception as ex:
                    print(ex)
                    break
         
        s_sql = "select brand from price_comparison"
        cursor.execute(s_sql)
        result = cursor.fetchall()
        print(result)
        
        conn.commit() 
        cursor.close()
        conn.close()
except Exception as ex:
    print("連線錯誤===",ex)

    

(1054, "Unknown column 'w' in 'field list'")
()


In [ ]:
print(insert)

In [5]:
for adx,datas in enumerate(data['card']):
    print(datas)

['MSI 微星 Gaming Radeon RX 5600 XT Boost Clock 顯卡： 1620 MHz 192 位元 6GB GDDR6 DP/HDMI 雙梅花型 3.0 風扇', 'MSI Gaming Radeon RX GDRR5 DirectX 12 VR Ready CFX 顯示卡', 'ZOTAC Gaming GeForce RTX 2060 6GB GDDR6 192 位元遊戲顯示卡 超輕巧型 ZT-T20600K-10M', 'ASUS ROG Strix AMD Radeon RX 5700XT Overclocked 8G GDDR6 HDMI DisplayPort Gaming Graphics Card (ROG-STRIX-RX5700XT-O8G-GAMING) 主機板', 'XFX HDMI 顯卡', 'Asus 華碩 GeForce GTX 1050 Ti 4GB 鳳凰風扇版 DVI-D HDMI DP 1.4 遊戲顯示卡 (PH-GTX1050TI-4G) 顯示卡', 'MSI Gaming Radeon RX GDRR5 DirectX 12 VR Ready CFX 顯示卡', 'Gigabyte 技嘉 Geforce GTX 1050 Ti 顯示卡 內建 4GB 記憶體 搭載風之力風扇 繪圖晶片（GV-N105TWF2OC-4GD）', 'MSI微星 Gaming GeForce RTX 3090 24GB GDRR6X 384-Bit HDMI/DP Nvlink Torx Fan 3 Ampere Architecture OC 顯示卡(RTX 3090 Ventus 3X 24G OC)', 'MSI微星 Gaming GeForce GT 710 2GB GDRR3 64-bit HDCP 支援 DirectX 12 OpenGL 4.5 單風扇低調顯卡(GT 710 2GD3 LP)', 'EVGA 06G-P4-1667-KR GeForce GTX 1660 Ti SC Ultra Gaming 6GB GDDR6,雙風扇', 'Sapphire 11265-67-20G Radeon Pulse RX 580 8GB GDDR5 雙 HDMI / 雙 DP OC 附背板 (UEFI) PCI-

In [47]:
insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s, %i,%s);"
                cursor.execute(insert,matchObj[0],every_data,adx+1,search_date)
        conn.commit() 

<class 'str'>


In [28]:
conn.close()
cursor.close()